In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import snapshot as sn
import h5py
from scipy.stats import binned_statistic_2d
import re

In [2]:
basePath = '../sims.TNG/TNG300-1/output/'
SnapNum = 99

In [3]:
simulation = (re.search(r'TNG(\d+-\d+)/', basePath)).group(1)
simulation

'300-1'

In [4]:
# Defining a function for unit conversion
def UnitConversion(array, unit):
    if unit == 'mass':
        array = array * 1e10 / h # In Msun unit
    if unit == 'length':
        array = array * a / (1e3 * h) # In Mpc unit
    return array

In [5]:
header = il.groupcat.loadHeader(basePath, SnapNum)
h = header['HubbleParam']
a = header['Time']
box_size = UnitConversion(header['BoxSize'], 'length')

In [6]:
def chunkPath(basePath, snapNum, chunkNum):
    snapPath = basePath + '/snapdir_' + str(snapNum).zfill(3) + '/'
    filePath = snapPath + 'snap_' + str(snapNum).zfill(3)
    filePath += '.' + str(chunkNum) + '.hdf5'
    return filePath

In [7]:
snapshot_path = sn.snapPath(basePath, 99)
snap_head = h5py.File(snapshot_path, 'r')
tot_chunks = snap_head['Header'].attrs['NumFilesPerSnapshot']
tot_chunks

600

In [8]:
mass_table = snap_head['Header'].attrs['MassTable']
mass_dm = UnitConversion(mass_table[1], 'mass')
mass_dm

58804657.49447129

In [9]:
nPixels = [600, 600]
minMax = [0, box_size]

In [10]:
net_grid = np.zeros(nPixels)

In [ ]:
for i in range(tot_chunks):
    filePath = chunkPath(basePath, SnapNum, i)
    with h5py.File(filePath, 'r') as data:
        header = dict(data['Header'].attrs.items())
        num_part = header['NumPart_ThisFile']
        x_dist = []
        y_dist = []
        mass = []
        
        if num_part[0] != 0:
            p0_x = UnitConversion(data['PartType0']['Coordinates'][:, 0], 'length')
            p0_y = UnitConversion(data['PartType0']['Coordinates'][:, 1], 'length')
            m0 = UnitConversion(data['PartType0']['Masses'][()], 'mass')
            x_dist.extend(p0_x)
            y_dist.extend(p0_y)
            mass.extend(m0)
        
        if num_part[1] != 0:
            p1_x = UnitConversion(data['PartType1']['Coordinates'][:, 0], 'length')
            p1_y = UnitConversion(data['PartType1']['Coordinates'][:, 1], 'length')
            m1 = np.full(num_part[1], mass_dm)
            x_dist.extend(p1_x)
            y_dist.extend(p1_y)
            mass.extend(m1)
        
        if num_part[4] != 0:
            p4_x = UnitConversion(data['PartType4']['Coordinates'][:, 0], 'length')
            p4_y = UnitConversion(data['PartType4']['Coordinates'][:, 1], 'length')
            m4 = UnitConversion(data['PartType4']['Masses'][()], 'mass')
            x_dist.extend(p4_x)
            y_dist.extend(p4_y)
            mass.extend(m4)
        
        if num_part[5] != 0:
            p5_x = UnitConversion(data['PartType5']['Coordinates'][:, 0], 'length')
            p5_y = UnitConversion(data['PartType5']['Coordinates'][:, 1], 'length')
            m5 = UnitConversion(data['PartType5']['Masses'][()], 'mass')
            x_dist.extend(p5_x)
            y_dist.extend(p5_y)
            mass.extend(m5)
        
        grid, _, _, _ = binned_statistic_2d(x_dist, y_dist, mass, 'sum', bins=nPixels, range=[minMax, minMax])
        net_grid += grid 
    print(i+1, end=' ')

1 

In [ ]:
pxSize = box_size / nPixels[0]
pxArea = pxSize**2

net_grid_density = net_grid / pxArea

In [ ]:
extent = [0, box_size, 0, box_size]

fig, ax = plt.subplots(figsize=(16,10))
plt.imshow(net_grid_density, cmap='magma', norm='log', extent=extent, aspect=nPixels[1]/nPixels[0])
ax.set_xlabel('x [Mpc]')
ax.set_ylabel('y [Mpc]')
ax.set_title(f'TNG{simulation}')
plt.colorbar(label='Mass Surface Density [log M$_\odot$ / Mpc$^{-2}$]')
plt.savefig(f'TNG{simulation}.png')
plt.show()